# Imports

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

---

# Carga de datos

In [2]:
features = pd.read_pickle("../../features/auctions_train_ventana_1.pkl")
targets = pd.read_pickle("../../targets/targets_ventana_1_auctions.pkl")
train = features.merge(targets, left_index=True, right_index=True)

---

# Preparación del set de datos para XGBoost

In [3]:
X, y = train.iloc[:,:-1], train.iloc[:,-1]
data_dmatrix = xgb.DMatrix(data=X,label=y)
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

In [11]:
xg_reg = xgb.XGBRegressor(colsample_bytree = 0.7, learning_rate = 0.5,
                n_estimators = 100, n_jobs=6, max_depth=7, objective="reg:linear")

In [12]:
xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.7, gamma=0, learning_rate=0.5, max_delta_step=0,
       max_depth=7, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=6, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [15]:
preds = xg_reg.predict(X_test.iloc[:100])
rmse = np.sqrt(mean_squared_error(y_test.iloc[:100], preds))
print("RMSE: %f" % (rmse))

RMSE: 89107.116607


---

# Pruebas

In [20]:
features_auctions = pd.read_pickle("../../features/auctions_train_ventana_1.pkl")
features_clicks = pd.read_pickle("../../features/clicks_train_ventana_1.pkl")

In [21]:
features_combinados_imputados = features_auctions.merge(features_clicks, right_index=True, left_index=True, suffixes=('_auctions', '_clicks')).fillna(0)

In [17]:
len(features_combinados_imputados)

9144

In [27]:
#features_auctions = pd.read_pickle("../../features/auctions_train_ventana_3.pkl")
targets = pd.read_pickle("../../targets/targets_ventana_1_auctions.pkl")
train = features_combinados_imputados.merge(targets, left_index=True, right_index=True)

X, y = train.iloc[:,:-1], train.iloc[:,-1]
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

dtrain = xgb.DMatrix(data=X_train, label=y_train)
dtest = xgb.DMatrix(data=X_test, label=y_test)

param = {'max_depth':6,
         'eta':0.3,
         'silent':1,
         'objective':'reg:linear',
         'eval_metric' : 'rmse',
         'obj' : 'reg:squarederror',
         'colsample_bytree' : 0.7,
         'min_child_weight' : 0.5,
         'random_state' : 0,
         'reg_alpha' : 0,
         'reg_lambda' : 1,
         'scale_pos_weight' : 1,
         'learning_rate' : 0.1}


num_round = 5000
stopping = 20
bst = xgb.train(param, dtrain, num_round, evals=[(dtrain, "train"), (dtest, "test")], early_stopping_rounds=stopping)
# make prediction
#preds = bst.predict(dtest)
#Stopping. Best iteration:
#[360]	train-rmse:67651	test-rmse:69665.3a

[0]	train-rmse:112057	test-rmse:113310
Multiple eval metrics have been passed: 'test-rmse' will be used for early stopping.

Will train until test-rmse hasn't improved in 20 rounds.
[1]	train-rmse:106220	test-rmse:107639
[2]	train-rmse:100664	test-rmse:102448
[3]	train-rmse:95998.8	test-rmse:98074.7
[4]	train-rmse:91925.2	test-rmse:94382.7
[5]	train-rmse:88440.8	test-rmse:91209.7
[6]	train-rmse:85339	test-rmse:88431.3
[7]	train-rmse:82728.4	test-rmse:86177.8
[8]	train-rmse:80499.4	test-rmse:84284.5
[9]	train-rmse:78494	test-rmse:82702.5
[10]	train-rmse:76818.4	test-rmse:81374.8
[11]	train-rmse:75350.1	test-rmse:80181.9
[12]	train-rmse:74086.8	test-rmse:79207
[13]	train-rmse:72958.6	test-rmse:78540.7
[14]	train-rmse:71949.1	test-rmse:77874.5
[15]	train-rmse:71099.3	test-rmse:77349.9
[16]	train-rmse:70341.1	test-rmse:76963.5
[17]	train-rmse:69703.8	test-rmse:76625.2
[18]	train-rmse:69078.7	test-rmse:76353
[19]	train-rmse:68561.6	test-rmse:76070.3
[20]	train-rmse:68129.1	test-rmse:75840.2

---

In [28]:
import lightgbm as lgb
#lgb_model = lgb.LGBMRegressor(boosting_type='dart', metric='rmse', n_estimators=50, colsample_bytree=0.9)
lgb_model = lgb.LGBMRegressor(boosting_type='dart', num_leaves=7, n_estimators=1000, metric='rmse',
                               learning_rate=0.05, colsample_bytree=0.2)
lgb_model.fit(X_train, y_train)
preds = lgb_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 74927.184424


In [16]:
preds_xgb = bst.predict(dtest)
preds_lgbm = lgb_model.predict(X_test)

In [29]:
p1 = 0.8
p2 = 0.2

preds = p1*preds_xgb + p2*preds_lgbm
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

NameError: name 'preds_xgb' is not defined

In [30]:
diferencias = abs((y_test.values - preds))

In [36]:
74000 /3600

20.555555555555557

---